Say you have a bag of 99 fair coins and 1 coin that has heads on both sides.  You draw a coin from the bag and flip it 10 times, each time showing heads.  What is the probability you drew the unfair coin?

# Analytic Solution
We want to know the probability the coin drawn from the bag was the unfair coin, given it flipped heads 10 times.  

---
## Conditional Probability
To solve this analytically, first we need to learn about conditional probability.  Consider two events we will represent using $A$ and $B$.  For our example, the events will be:   

        A) drawing the unfair coin from the bag    
        B) flipping heads 10 times 

The probability of both events happening, $P(A\cap B)$, is equal to the probability of $A$ happening, $P(A)$, multiplied by the probability of $B$ happening given $A$ has occurred, $P(B|A)$.  

$$P(A\cap B) = P(B|A) P(A)$$

We could also switch this around, as $P(A\cap B)$ also is equal to the probability of $B$ happening, $P(B)$, multiplied by the probability of $A$ happening given $B$ has occurred, $P(A|B)$.

$$P(A\cap B) = P(A|B) P(B)$$

Putting these equalities together, we have 
$$P(A|B) P(B) = P(B|A) P(A)$$

which we can rewrite as
$$ P(A | B) = \frac{P(B | A) P(A)}{P(B)} $$

This is Bayes' theorem.

---
## What about the coin you drew?
To figure out the probability you drew the unfair coin, we have
- the probability of drawing the unfair coin from the bag: $P(A)$
- the probability of flipping heads 10 times: $P(B)$

We know the probability of drawing the unfair coin.
$$P(A) = 1 / 100$$

If we did draw the unfair coin, we know the probability of flipping heads 10 times will be

$$P(B | A) = 10 / 10 = 1$$

What about the probabilities related to drawing a fair coin, or not drawing the unfair coin, $P(!A)$?  The probability of not drawing the fair coin is $1 - P(A)$, or

$$P(!A) = 99 / 100$$

If we did draw a fair coin, we know the probability of flipping heads will be 1/2 for each flip, so for 10 flips we get

$$P(B | !A) = 1/2^{10}$$

From this we can calculate the probability of flipping nheads 10 times, $P(B)$, as

$$P(B) = P(B | A) P(A) + P(B | !A) P(!A)$$

or 

$$P(B) = 1 \times 1/100 + 1/2^{10} \times 99/100$$

and we have everything we need to solve the problem.

In [1]:
p_a = 1.0 / 100.0
p_ba = 1.0
p_na = 99.0 / 100.0
p_bna = 0.5 ** 10

p_b = p_ba * p_a + p_bna * p_na
p_ab = p_ba * p_a / p_b

print(f'Probability the drawn coin was unfair, given it flipped heads 10 times: {p_ab:0.3}')

Probability the drawn coin was unfair, given it flipped heads 10 times: 0.912


So the coin you drew is 91% likely to be the unfair coin.  **Is that as lower/higher than you thought?**

# Calculated Solution
Lets apply our Python hacking skills to instead simulate the experiment and see how often flipping 10 heads resulted from the unfair coin.  Inspired by [Jake Vanderplas'](http://vanderplas.com/) PyCon 2016 talk "[Statistics for Hacker](https://www.youtube.com/watch?v=Iq9DzN6mvYA)".

In [2]:
import numpy as np
np.set_printoptions(suppress=True)

In [3]:
import time

In [4]:
import bokeh.plotting
from bokeh.palettes import Category10_10 as palette

In [5]:
import bokeh.resources
bokeh.plotting.output_notebook(resources=bokeh.resources.INLINE)

Loading BokehJS ...

Setup the coins:

In [6]:
# 100 coin all fair
n_coins = 100
coins = np.zeros(n_coins, dtype=np.int)  

# make one random coin unfair
coins[np.random.randint(1, n_coins, 1)] = 1

Define the coin draw-flip routine:

In [7]:
def flip_generator2(coins, *, n_flips=10):
    n_coins = len(coins)
    
    coin_index = np.random.randint(0, n_coins, 1)[0]
    drawn_coin = coins[coin_index]
    coin_unfair = drawn_coin == 1  # did we get the unfair coin?

    if coin_unfair:
        all_heads = True
    else: 
        for j in range(n_flips):
            flip = np.random.rand()  # random number between [0, 1)

            if flip < 0.5:
                all_heads = True
            else:
                all_heads = False
                break

    if all_heads:
        # report the index and if the coin was unfair
        yield coin_unfair

Simulate the coin draw-flip routine a million times.

In [23]:
n_all_heads = 0
n_unfair = 0
all_heads = []
all_unfair = []

for i in range(int(3e6)):
    
    res = list(flip_generator2(coins))
    if res:
        n_all_heads += 1
        if res[0]:
            n_unfair += 1
        frac_unfair = n_unfair / n_all_heads
        all_heads.append(n_all_heads)
        all_unfair.append(frac_unfair)

In [25]:
print(f'Of {n_all_heads} all-heads occurances, {frac_unfair*100:.2f}% were unfair.')

Of 32807 all-heads occurances, 91.31% were unfair.


**This is the same probability we got from the analytic solution.**  Lets see how this changed over the course of the simulation.

In [29]:
all_heads[0]

1

In [30]:
p = bokeh.plotting.figure(width=400, height=400,
    y_axis_label='fraction of times 10 heads was from the unfair coin',
    x_axis_label='trials (e6)')

p.line(all_heads, all_unfair, 
       color=palette[0], legend='simulated result')
p.line([all_heads[0], all_heads[-1]], [p_ab, p_ab], 
       color=palette[1], legend='analytic result')

p.legend.location = 'top_right'
bokeh.plotting.show(p)

We see the simulated result equilibrated rather quickly to a value close to the analytic result.  We expect that running for a longer number of iterations would further improve the agreement.

In [ ]:
my_figure = bokeh.plotting.figure(plot_width=800, plot_height=400)
test_data = bokeh.models.sources.ColumnDataSource(data=dict(x=[0], y=[0]))
line = my_figure.line("x", "y", source=test_data)
handle = bokeh.io.show(my_figure, notebook_handle=True)

new_data=dict(x=[0], y=[0])
x = []
y = []

step = 0
n_all_heads = 0
n_unfair = 0

max_step = 1000000  # arbitrary stop point for example
n_show = 500  # number of points to keep and show
while step < max_step:
    step += 1

    res = list(flip_generator2(coins))
    if res:
        n_all_heads += 1
        if res[0]:
            n_unfair += 1
        frac_unfair = n_unfair / n_all_heads

        new_data['x'] = [n_all_heads]  
        new_data['y'] = [frac_unfair]  
        
        test_data.stream(new_data, n_show)

        bokeh.io.push_notebook(handle=handle)
        time.sleep(0.1)